In [17]:
import open3d
import numpy as np
import pandas as pd
import os
import tqdm

In [2]:
def preprocess_pcd(pcd, voxel_size):
    pcd_down = open3d.voxel_down_sample(pcd, voxel_size)

    radius_normal = voxel_size * 2
    open3d.geometry.estimate_normals(pcd_down, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = open3d.registration.compute_fpfh_feature(pcd_down, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh


def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    result = open3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        open3d.registration.TransformationEstimationPointToPoint(False), 4, 
        [open3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        open3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result


def refine_registration(source, target, voxel_size, t):
    distance_threshold = voxel_size * 0.4
    result = open3d.registration.registration_icp(source, target, distance_threshold, t, open3d.registration.TransformationEstimationPointToPlane())
    return result

def read_pcd_file(file_path, voxel_size):
    pcd = open3d.io.read_point_cloud(file_path)
    pcd = open3d.geometry.voxel_down_sample(pcd, voxel_size)
    return pcd

In [24]:
data_dir = "data/DatasetV2/Sequence/04"

pcd_files = os.listdir(data_dir)
pcd_files = sorted(pcd_files, key=lambda f: int(f.split(".")[0].split("_")[1]))

In [25]:
source = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[0]))
target = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[1]))

source, source_fpfh = preprocess_pcd(source, 0.05)
target, target_fpfh = preprocess_pcd(target, 0.05)

source.paint_uniform_color([0.906, 0.298, 0.236])
target.paint_uniform_color([0.204, 0.596, 0.859])

ransac_result = execute_global_registration(source, target, source_fpfh, target_fpfh, 0.05)

source.transform(ransac_result.transformation)

open3d.draw_geometries([source, target])

In [30]:
pcds = []
n_samples = 3

for i in range(len(pcd_files) // n_samples):

    source = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[n_samples * i]))

    for j in range(n_samples):
        target = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[n_samples * i + j + 1]))

        source, source_fpfh = preprocess_pcd(source, 0.05)
        target, target_fpfh = preprocess_pcd(target, 0.05)

        source.paint_uniform_color([0.906, 0.298, 0.236])
        target.paint_uniform_color([0.204, 0.596, 0.859])

        ransac_result = execute_global_registration(source, target, source_fpfh, target_fpfh, 0.05)
        
        icp_result = refine_registration(source, target, 0.05, ransac_result.transformation)

        source.transform(icp_result.transformation)
        
        source = source + target
    
    source.paint_uniform_color([0.906, 0.298, 0.236])
    # open3d.draw_geometries([source])
    
    pcds.append(source)

In [32]:
for i in range(len(pcds) - 1):
    source, source_fpfh = preprocess_pcd(pcds[i], 0.05)
    target, target_fpfh = preprocess_pcd(pcds[i + 1], 0.05)

    source.paint_uniform_color([0.906, 0.298, 0.236])
    target.paint_uniform_color([0.204, 0.596, 0.859])

    ransac_result = execute_global_registration(source, target, source_fpfh, target_fpfh, 0.05)
    
    icp_result = refine_registration(source, target, 0.05, ransac_result.transformation)

    source.transform(icp_result.transformation)
    
    open3d.draw_geometries([source, target])

In [ ]:
output_dir = data_dir.replace("Sequence", "SequenceCombined")

if not os.path.exists(output_dir): os.mkdir(output_dir)

for i, pcd in enumerate(pcds):
    open3d.io.write_point_cloud(os.path.join(output_dir, f"seq_{data_dir[-2:]}_{i}.pcd"), pcd)

In [19]:
def make_pcd(points, color=None):
    c = [1, 0, 0] if color is None else color
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(points)
    pcd.colors = open3d.utility.Vector3dVector([c for i in range(len(points))])
    return pcd

def get_limits(pcd):
    pcd_points = np.asarray(pcd.points)

    x_min, y_min, z_min = np.min(pcd_points, axis=0)
    x_max, y_max, z_max = np.max(pcd_points, axis=0)

    return x_min, x_max, y_min, y_max, z_min, z_max

def get_grid(pcd, cell_size):
    x_min, x_max, y_min, y_max, z_min, z_max = get_limits(pcd)
    y_val = np.mean([y_min, y_max])

    points = []
    x_n = int((x_max - x_min) // cell_size)
    z_n = int((z_max - z_min) // cell_size)
    for i in range(z_n):
        z0 = float(z_min + cell_size * (i + 1))
        for j in range(x_n):
            x0 = float(x_min + cell_size * (j + 1))
            points.append([x0, y_val, z0])

    return points

def get_features(feature_file):
    data = np.load(feature_file)
    scores = data["scores"]
    features = open3d.registration.Feature()
    features.data = data["features"].T
    keypts = open3d.geometry.PointCloud()
    keypts.points = open3d.utility.Vector3dVector(data["keypts"])
    return keypts, features, scores

def filter_indices(points, p, cell_size):
    px_min = p[0] - cell_size
    px_max = p[0] + cell_size
    pz_min = p[2] - cell_size
    pz_max = p[2] + cell_size
    xf = np.logical_and(points[:, 0] > px_min, points[:, 0] < px_max)
    zf = np.logical_and(points[:, 2] > pz_min, points[:, 2] < pz_max)
    return np.logical_and(xf, zf)

def get_cell_features(feature_file, p, cell_size):
    data = np.load(feature_file)
    scores = data["scores"]

    f = filter_indices(data["keypts"], p, cell_size)
    f = np.where(f)[0]

    features = open3d.registration.Feature()
    features.data = data["features"][f].T

    keypts = open3d.geometry.PointCloud()
    keypts.points = open3d.utility.Vector3dVector(data["keypts"][f])

    return keypts, features, scores

def registration(src_keypts, tgt_keypts, src_desc, tgt_desc, distance_threshold):
    result = open3d.registration_ransac_based_on_feature_matching(
        src_keypts, tgt_keypts, src_desc, tgt_desc,
        distance_threshold,
        open3d.TransformationEstimationPointToPoint(False), 3,
        [open3d.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         open3d.CorrespondenceCheckerBasedOnDistance(distance_threshold),
         open3d.CorrespondenceCheckerBasedOnNormal(normal_angle_threshold=np.deg2rad(30.0))],
        open3d.RANSACConvergenceCriteria(40000000, 600))
    return result

def print_registration_result(src_keypts, tgt_keypts, result_ransac, end="\n"):
    print(f"Keypts: [{len(src_keypts.points)}, {len(tgt_keypts.points)}]", end="\t")
    print(f"No of matches: {len(result_ransac.correspondence_set)}", end="\t")
    print(f"Inlier RMSE: {result_ransac.inlier_rmse:.4f}", end=end)
    
def create_camera(camera_width, color):
    points = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1], [0, 1, 1], [1, 1, 1]])
    points = points * camera_width
    points = points - camera_width / 2
    lines = [[0, 1], [0, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 7], [6, 7], [0, 4], [1, 5], [2, 6], [3, 7]]
    colors = [color for i in range(len(lines))]
    line_set = open3d.geometry.LineSet()
    line_set.points = open3d.utility.Vector3dVector(points)
    line_set.lines = open3d.utility.Vector2iVector(lines)
    line_set.colors = open3d.utility.Vector3dVector(colors)
    return line_set

def rgb(r, g, b):
    return r / 255.0, g / 255.0, b / 255.0

In [20]:
cell_size = 3

tgt_pcd_file = "data/DatasetV2/Primary/07/lidar_1637299421190934300.pcd"
tgt_file_name = tgt_pcd_file.split('/')[-1]
tgt_feature_file = f"data/FeaturesV8/0.05/{tgt_file_name.replace('pcd', 'npz')}"

pcd = open3d.read_point_cloud(tgt_pcd_file)
pcd = open3d.voxel_down_sample(pcd, 0.025)
pcd.paint_uniform_color(rgb(149, 165, 166))

grid_points = get_grid(pcd, cell_size)
grid = make_pcd(grid_points)

In [23]:
sequence_id = 4
sequence_dir = "data/DatasetV2/SequenceCombined/{:02d}".format(sequence_id)

for seq_file_name in os.listdir(sequence_dir):
    src_feature_file = os.path.join("data/FeaturesV8/0.05/", seq_file_name.replace("pcd", "npz"))
    best_score = 0
    best_p = None
    t = None

    for p in tqdm.notebook.tqdm(grid_points):
        src_keypts, src_features, src_scores = get_features(src_feature_file)
        tgt_keypts, tgt_features, tgt_scores = get_cell_features(tgt_feature_file, p, cell_size)

        if len(tgt_keypts.points) < 2000:
            continue

        src_keypts.paint_uniform_color([1, 0.706, 0])
        tgt_keypts.paint_uniform_color([0, 0.651, 0.929])

        result_ransac = registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)
        
        # print_registration_result(src_keypts, tgt_keypts, result_ransac, end="\n")

        if best_score < len(result_ransac.correspondence_set):
            best_score = len(result_ransac.correspondence_set)
            best_p = p
            t = result_ransac.transformation
            
    if best_p is not None:
        src_keypts, src_features, src_scores = get_features(src_feature_file)

        src_keypts.paint_uniform_color([1, 0.706, 0])

        src_camera = create_camera(camera_width=0.25, color=[1, 0, 0])

        result = open3d.registration.registration_icp(
            src_keypts, pcd, 0.05, t, open3d.registration.TransformationEstimationPointToPoint()
        )

        src_keypts.transform(result.transformation)
        src_camera.transform(result.transformation)
        open3d.visualization.draw_geometries([src_keypts, pcd, src_camera, grid])


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]